In [1]:
import cobrakbase

cobrakbase 0.2.7


In [3]:
%%time
modelseed_local = cobrakbase.modelseed.from_local('/Users/fliu/workspace/jupyter/ModelSEEDDatabase')

CPU times: user 16.1 s, sys: 201 ms, total: 16.3 s
Wall time: 16.4 s


In [64]:
index = {}
for rxn_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    cs = rxn.cstoichiometry
    for p in cs:
        if p[0] not in index:
            index[p[0]] = set()
        index[p[0]].add(rxn_id)

In [72]:
comps = {}
rxn_len = {}
n_obs = set()
for rxn_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    if not rxn.is_obsolete:
        n_obs.add(rxn_id)
    cs = rxn.cstoichiometry
    c = len(set(map(lambda x: x[1], cs)))
    if c not in comps:
        comps[c] = set()
    comps[c].add(rxn_id)
    c = len(cs)
    if c not in rxn_len:
        rxn_len[c] = set()
    rxn_len[c].add(rxn_id)


In [77]:
res = (index['cpd07274'] & index['cpd16503'] & comps[2] & n_obs)
for rxn_id in res:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    print(rxn)
    print(rxn.data['definition'])

rxn47815: 2.0 cpd00067 [1] + cpd16503 [0] + 2.0 cpd27746 [0] <=> 4.0 cpd00067 [0] + cpd07274 [0] + 2.0 cpd27884 [0]
(2) H+[1] + (1) Plastoquinol-9[0] + (2) Oxidized-Plastocyanins[0] <=> (4) H+[0] + (1) Plastoquinone A[0] + (2) Plastocyanin-Reduced[0]
rxn46370: cpd00007 [0] + 4.0 cpd00067 [0] + 2.0 cpd16503 [0] <=> 2.0 cpd00001 [0] + 4.0 cpd00067 [1] + 2.0 cpd07274 [0] + cpd11632 [0]
(1) O2[0] + (4) H+[0] + (2) Plastoquinol-9[0] <=> (2) H2O[0] + (4) H+[1] + (2) Plastoquinone A[0] + (1) hn[0]
rxn40737: 4.0 cpd00067 [0] + cpd07274 [0] + 2.0 cpd12713 [0] <=> 2.0 cpd00067 [1] + cpd16503 [0]
(4) H+[0] + (1) Plastoquinone A[0] + (2) e-[0] <=> (2) H+[1] + (1) Plastoquinol-9[0]
crxn00002: cpd16503 [1] + 2.0 cpd12265 [0] <=> 2.0 cpd00067 [0] + 2.0 cpd12239 [0] + cpd07274 [1]
(1.0) Plastoquinol-9[1] + (2.0) Oxidized plastocyanin[0] <=> (2.0) H+[0] + (2.0) Reduced plastocyanin[0] + (1.0) Plastoquinone A[1]


In [73]:
res = (index['cpd00002'] & index['cpd00008'] & index['cpd00067'] & comps[2] & rxn_len[6] & n_obs)
for rxn_id in res:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    print(rxn)

rxn08173: cpd00008 [0] + cpd00009 [0] + 4.0 cpd00067 [1] --> cpd00001 [0] + cpd00002 [0] + 3.0 cpd00067 [0]
rxn30646: cpd00008 [0] + cpd00009 [0] + 5.0 cpd00067 [0] <-- cpd00001 [0] + cpd00002 [0] + 4.0 cpd00067 [1]
rxn31585: 3.0 cpd00008 [0] + 3.0 cpd00009 [0] + 14.0 cpd00067 [1] <-- 3.0 cpd00001 [0] + 3.0 cpd00002 [0] + 11.0 cpd00067 [0]
rxn34197: cpd00002 [1] + cpd00008 [0] + 3.0 cpd00067 [0] <=> cpd00002 [0] + cpd00008 [1] + 3.0 cpd00067 [1]
rxn10443: cpd00001 [0] + cpd00002 [0] + cpd00067 [0] --> cpd00008 [0] + cpd00009 [0] + 2.0 cpd00067 [1]
rxn29496: cpd00001 [0] + cpd00002 [0] + cpd00067 [1] --> cpd00008 [0] + cpd00009 [0] + 2.0 cpd00067 [0]
rxn09528: cpd00008 [0] + cpd00009 [0] + 3.0 cpd00067 [1] <-- cpd00001 [0] + cpd00002 [0] + 2.0 cpd00067 [0]
rxn09780: cpd00002 [1] + cpd00008 [0] + cpd00067 [0] <=> cpd00002 [0] + cpd00008 [1] + cpd00067 [1]
rxn05145: cpd00001 [0] + cpd00002 [0] + cpd00009 [1] --> cpd00008 [0] + 2.0 cpd00009 [0] + cpd00067 [0]


In [17]:
import pandas as pd
import copy

In [7]:
df = pd.read_csv('../data/custom_reactions.tsv', sep='\t')

In [23]:
rxns = {}
for row_id, d in df.iterrows():
    rxn_id = d['rxn_id']
    name = d['Name']
    cpd_id = d['cpd_id']
    coeff = d['coeff']
    cmp_token = d['cmp']
    if rxn_id not in rxns:
        rxns[rxn_id] = copy.deepcopy(base)
        rxns[rxn_id]['metabolites'] = {}
    rxns[rxn_id]['id'] = rxn_id
    rxns[rxn_id]['name'] = name
    rxns[rxn_id]['abbreviation'] = rxn_id
    cc = (cpd_id, cmp_token)
    if cc not in rxns[rxn_id]['metabolites']:
        rxns[rxn_id]['metabolites'][cc] = coeff

In [45]:
import math
def to_stoichiometry(cs):
    l = []
    for p in cs:
        cpd = modelseed_local.get_seed_compound(p[0])
        ss = '{}:{}:{}:{}:"{}"'.format(cs[p], p[0], p[1], '0', cpd.name)
        l.append(ss)
    return ";".join(l)
def to_equation(cs):
    lhs = []
    rhs = []
    for p in cs:
        coeff = cs[p]
        if coeff < 0:
            lhs.append("({}) {}[{}]".format(math.fabs(coeff), p[0], p[1]))
        else:
            rhs.append("({}) {}[{}]".format(math.fabs(coeff), p[0], p[1]))
    return "{} <=> {}".format(' + '.join(lhs), ' + '.join(rhs))
def to_definition(cs):
    lhs = []
    rhs = []
    for p in cs:
        coeff = cs[p]
        cpd = modelseed_local.get_seed_compound(p[0])
        if coeff < 0:
            lhs.append("({}) {}[{}]".format(math.fabs(coeff), cpd.name, p[1]))
        else:
            rhs.append("({}) {}[{}]".format(math.fabs(coeff), cpd.name, p[1]))
    return "{} <=> {}".format(' + '.join(lhs), ' + '.join(rhs))

In [57]:
for rxn_id in rxns:
    cs = rxns[rxn_id]['metabolites']
    rxns[rxn_id]['code'] = to_equation(cs)
    rxns[rxn_id]['equation'] = to_equation(cs)
    rxns[rxn_id]['stoichiometry'] = to_stoichiometry(cs)
    rxns[rxn_id]['definition'] = to_definition(cs)
    rxns[rxn_id]['compound_ids'] = ';'.join(set(map(lambda x: x[0], cs)))
    rxns[rxn_id]['linked_reaction'] = ''
    rxns[rxn_id]['pathways'] = 'MetaCyc: Photosynthesis light reactions'
    rxns[rxn_id]['aliases'] = ''
    modelseed_local.reactions[rxn_id] = copy.deepcopy(rxns[rxn_id])

In [62]:
rxn = modelseed_local.get_seed_reaction('crxn00001')
rxn.cstoichiometry

{('cpd00067', '0'): -4.0,
 ('cpd11632', '1'): -4.0,
 ('cpd07274', '3'): -2.0,
 ('cpd16503', '3'): 2.0,
 ('cpd00001', '2'): -2.0,
 ('cpd00007', '2'): 1.0,
 ('cpd00067', '2'): 4.0}

In [12]:
base = {
    'id': 'rxn00001',
    'abbreviation': 'R00004',
    'name': 'diphosphate phosphohydrolase',
    'code': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0]',
    'stoichiometry': '-1:cpd00001:0:0:"H2O";-1:cpd00012:0:0:"PPi";2:cpd00009:0:0:"Phosphate";1:cpd00067:0:0:"H+"',
    'equation': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0] + (1) cpd00067[0]',
    'definition': '(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] + (1) H+[0]',
    'is_transport': 0,
    'reversibility': '>',
    'direction': '=',
    'abstract_reaction': None,
    'pathways': 'MetaCyc: Degradation (Degradation/Utilization/Assimilation); Glyphosate-Degradation (glyphosate degradation); Noncarbon-Nutrients (Inorganic Nutrient Metabolism); PWY-7805 ((aminomethyl)phosphonate degradation); PWY-7807 (glyphosate degradation III); Phosphorus-Compounds (Phosphorus Compound Metabolism)',
    'aliases': 'AraCyc: INORGPYROPHOSPHAT-RXN|BiGG: IPP1; PPA; PPA_1; PPAm|BrachyCyc: INORGPYROPHOSPHAT-RXN|KEGG: R00004|MetaCyc: INORGPYROPHOSPHAT-RXN|Name: Diphosphate phosphohydrolase; Inorganic diphosphatase; Inorganic pyrophosphatase; Pyrophosphate phosphohydrolase; diphosphate phosphohydrolase; inorganic diphosphatase; inorganic diphosphatase (one proton translocation); inorganicdiphosphatase; pyrophosphate phosphohydrolase',
    'ec_numbers': '3.6.1.1',
    'deltag': -3.46,
    'deltagerr': 0.05,
    'compound_ids': 'cpd00001;cpd00009;cpd00012;cpd00067',
    'status': 'OK',
    'is_obsolete': 0,
    'linked_reaction': 'rxn27946;rxn27947;rxn27948;rxn32487;rxn38157;rxn38158',
    'notes': 'GCC|HB|EQC|EQU',
    'source': 'Primary Database'
}

In [56]:
rxns

{'crxn00001': {'id': 'crxn00001',
  'abbreviation': 'crxn00001',
  'name': 'Photosystem II',
  'code': '(4.0) cpd00067[0] + (4.0) cpd11632[1] + (2.0) cpd07274[3] + (2.0) cpd00001[2] <=> (2.0) cpd16503[3] + (1.0) cpd00007[2] + (4.0) cpd00067[2]',
  'stoichiometry': '-4:cpd00067:0:0:"H+";-4:cpd11632:1:0:"hn";-2:cpd07274:3:0:"Plastoquinone A";2:cpd16503:3:0:"Plastoquinol-9";-2:cpd00001:2:0:"H2O";1:cpd00007:2:0:"O2";4:cpd00067:2:0:"H+"',
  'equation': '(4.0) cpd00067[0] + (4.0) cpd11632[1] + (2.0) cpd07274[3] + (2.0) cpd00001[2] <=> (2.0) cpd16503[3] + (1.0) cpd00007[2] + (4.0) cpd00067[2]',
  'definition': '(4.0) H+[0] + (4.0) hn[1] + (2.0) Plastoquinone A[3] + (2.0) H2O[2] <=> (2.0) Plastoquinol-9[3] + (1.0) O2[2] + (4.0) H+[2]',
  'is_transport': 0,
  'reversibility': '>',
  'direction': '=',
  'abstract_reaction': None,
  'pathways': 'MetaCyc: Photosynthesis light reactions',
  'aliases': '',
  'ec_numbers': '3.6.1.1',
  'deltag': -3.46,
  'deltagerr': 0.05,
  'compound_ids': 'cpd07274